# Metrics and loss

> These functions are taken mostly from sakitools, which is developed by Fabian hickert 

In [ ]:
#| default_exp metrics_and_loss

In [ ]:
#| export
import tensorflow as tf

2023-04-13 01:57:52.756711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 01:57:53.062096: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
#| export
def dice_coef(y_true, y_pred, smooth=0):        
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection =  tf.keras.backend.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / ( tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)
    return dice


In [ ]:
#| export
def tversky_index(alpha=0.5, beta=0.5, gamma=1.5,name=None):
    def TI(y_true, y_pred):
        y_true = tf.keras.backend.flatten(y_true)
        y_pred = tf.keras.backend.flatten(y_pred)
    
        true_positives = tf.keras.backend.sum(y_true * y_pred)
        false_positives = tf.keras.backend.sum(y_pred * (1 - y_true))
        false_negatives = tf.keras.backend.sum((1 - y_pred) * y_true)

        numerator = true_positives + 1e-7
        denominator = true_positives + alpha * false_negatives + beta * false_positives + 1e-7
        tversky_index = numerator / denominator
        return tversky_index
    if name is None:
        TI.__name__ = "TI_%.02f_%.02f_%.02f" % (alpha, beta, gamma)
    else:
        TI.__name__ = name
    return TI


class FNR(tf.keras.metrics.Recall):
    """! FNR - False negative rate metric
             FN / (FN + TP)
    """
    def __init__(self, thresholds=0.5, name="fnr_escapee", **kwargs):
        super().__init__(name=name, thresholds=thresholds, **kwargs)
        
    def result(self):
        return 1.0 - super().result()
    
    
class FDR(tf.keras.metrics.Precision):
    """! FDR - False discovery rate metric
            FP / (FP + TP)
    """
    def __init__(self, thresholds=0.5, name="fdr_overreject", **kwargs):
        super().__init__(name=name, thresholds=thresholds, **kwargs)
        
    def result(self):
        return 1.0 - super().result()
        
    
#
# Loss Functions
#

def dice_coef_loss(y_true, y_pred):
    #return 1 - tf.keras.backend.log(dice_coef(y_true, y_pred))
    return 1 - dice_coef(y_true, y_pred)

def dice_loss(y_true, y_pred):
    return dice_coef_loss(y_true, y_pred)

def tversky_focal_loss(alpha=0.5, beta=0.5, gamma=1.5, name=None):
    def TFL(y_true, y_pred):
        ti = tversky_index(alpha, beta, gamma)
        return tf.keras.backend.pow(1 - ti(y_true, y_pred), gamma)
    if name is None:
        TFL.__name__ = "TFL_%.02f_%.02f_%.02f" % (alpha, beta, gamma)
    else:
        TFL.__name__ = name
    
    return TFL

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()